# Deployement of Document Extraction and Classification via importing the previously trained and exported models.

In [22]:
#pred_sentences=["","XE Money Transfer is a Good money transfer service (7.5/10) recommended by Monito. It's a fully licensed and authorized money transfer provider, and has a very high number and percentage of positive customer reviews on TrustPilot (10/10). Setting up your transfer with XE Money Transfer is quick and easy (8.3/10). However, the exchange rate margin makes up most of the total cost of your transfer with XE Money Transfer service (3.8/10)."]
#pred_sentences=["","Fraud is new to the Top Ten list, with many consumers complaining about a range of scams including work-at-home schemes and fraudulent sweepstakes and lotteries. One man from Ohio reported being thrilled when someone called to tell him he'd won $850,000 in a sweepstakes. All he had to do to claim the prize was transfer $220 to someone in Jamaica to pay the fees on his winnings. After he transferred that money, they asked him for another $650. It was after that the Ohio man discovered that the person was not really from a sweepstakes company and he was not really a winner. \"Fraud is an especially challenging problem because scammers often target U.S. consumers from foreign countries, making law enforcement difficult,\" said Anna Huddleston-Aycock, President of North American Consumer Protection Investigators."]
#pred_sentences=["","My password is uhiudbdk and also pls transfer 200 to it. You can take my email id: XXXX. Details of my credit card:XXXX. Very important document. COnfidential. Take note."]
#pred_sentences=["","For the sample from materials sciences, directed at an internal fellowship, the one-page essay has an especially difficult task: The writer must persuade those who already know him (and thus know both his strengths and limitations) that he is worthy of internal funds to help him continue his graduate education. He attempts this by first citing the specific goal of his research group, followed by a brief summary of the literature related to this topic, then ending with a summary of his own research and lab experience."]
imp=["confidential","internal use only", "not for distribution", "not for public distribution", "classified ", "password","passwords","email","important","very important","database","atm pin","cvv","mpin","dataset"]
#len(pred_sentences[1])

### Downloading the trained bert estimator model

In [2]:
#from google.colab import drive
#drive.mount('/content/gdrive')

!gdown --id 1yLJnjYNjranb5zAXXr_q883trF1aAxZx

Downloading...
From: https://drive.google.com/uc?id=1yLJnjYNjranb5zAXXr_q883trF1aAxZx
To: /content/bert-model2.zip
408MB [00:03, 122MB/s]


### Downloading the saved LSTM keras model

In [3]:
#!cp "/content/gdrive/My Drive/model2.h5" "/content"

!gdown --id 1--kvG6PlwLSmD834J0Bw9Fsa7efK8FJk

Downloading...
From: https://drive.google.com/uc?id=1--kvG6PlwLSmD834J0Bw9Fsa7efK8FJk
To: /content/model2.h5
4.24MB [00:00, 132MB/s]


In [4]:
!unzip /content/bert-model2.zip

Archive:  /content/bert-model2.zip
   creating: bert_news_category/1597156650/
  inflating: bert_news_category/1597156650/saved_model.pb  
   creating: bert_news_category/1597156650/variables/
  inflating: bert_news_category/1597156650/variables/variables.index  
  inflating: bert_news_category/1597156650/variables/variables.data-00000-of-00001  
   creating: bert_news_category/1597156650/assets/
  inflating: bert_news_category/1597156650/assets/vocab.txt  


In [1]:
%tensorflow_version 1.x magic

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x magic`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


Loading BERT model

In [2]:
from tensorflow.contrib import predictor
predict_fn = predictor.from_saved_model('/content/bert_news_category/1597156650')

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /content/bert_news_category/1597156650/variables/variables


In [3]:
import pandas as pd
import numpy as np
np.random.seed(1337)
from keras import Sequential
from keras.utils import Sequence
from keras.layers import LSTM, Dense, Masking
import numpy as np
import keras
from keras.utils import np_utils
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Embedding, Dense, Input, concatenate, Layer, Lambda, Dropout, Activation
import datetime
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, TensorBoard
import tensorflow as tf
import tensorflow_hub as hub

Using TensorFlow backend.


In [4]:
from keras.models import load_model
!pip install h5py==2.10.0

Loading LSTM keras model

In [5]:
model = load_model('/content/model2.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



### Function for splitting the document data into smaller chunks, as the BERT transformer model is limited to 512 tokens.

In [6]:
def get_split(text1):
  l_total = []
  l_parcial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:200]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*150:w*150 + 200]
      l_total.append(" ".join(l_parcial))
  return l_total

In [7]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

In [8]:
import re
def clean_txt(text):
  text = re.sub("'", "",text)
  text=re.sub("(\\W)+"," ",text)    
  return text

In [9]:
lab = np.array([0,1, 2, 3, 4, 5,6,7,8,9])
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'
# The list containing all the classes (train['SECTION'].unique())
label_list = [x for x in np.unique(lab)]
label_list

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [10]:
MAX_SEQ_LENGTH=200

In [11]:
!pip install bert-tensorflow==1.0.1
!pip install textract
!pip install PyPDF2

     |████████████████████████████████| 71kB 2.5MB/s 
     |████████████████████████████████| 32.8MB 131kB/s 
     |████████████████████████████████| 112kB 27.7MB/s 
     |████████████████████████████████| 8.9MB 16.7MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 5.6MB 18.4MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
     |████████████████████████████████| 112kB 50.3MB/s 
     |████████████████████████████████| 153kB 50.9MB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 112kB 52.9MB/s 
     |████████████████████████████████| 1.9MB 12.7MB/s 
  Created wheel for EbookLib: filename=EbookLib-0.17.1-cp37-none-any.whl size=38187 sha256=700e3a7143f97062476b256a1090da62f2fffd99c904aec2af6c955ce3f74ef7
  Stored in directory: /root/.cache/pip/wheels/84/11/01/951369cbbf8f96878786a1f4da68bd7ac19a5d945b38e03d54
  Created wheel for python-pptx: filename=python_pptx-0.6.18-cp37-none-

     |████████████████████████████████| 81kB 2.9MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp37-none-any.whl size=61102 sha256=208ba12ba260be233e56adb56ad4d8feed239984a8e25a21fecf0894a490ad5d
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [12]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [13]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [14]:
# A method to get predictions
def getPrediction(in_sentences,predict_fn, type_output = "features"):
  #A list to map the actual labels to the predictions
  labels = lab
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  #Predicting the classes 
  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in input_features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)
  pred_dict = {
    'input_ids': all_input_ids,
    'input_mask': all_input_mask,
    'segment_ids': all_segment_ids,
    'label_ids': all_label_ids
    }
  #predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  #print(type(predict_input_fn))
  predictions = predict_fn(pred_dict)
  print([predictions['pooled_output']])
  #print([enumerate(predictions)])
  #print([pooled_output for prediction in enumerate(predictions['pooled_output'])])
 # print([pooled_output for prediction in enumerate(predictions['pooled_output'])].shape)
  if type_output == "features":
    return predictions['pooled_output']
  else:
    return [
        (sentence, prediction, label)
        for sentence, prediction, label in zip(pred_sentences, predictions['probabilities'], predictions['labels'])
    ]

In [38]:
def val_generator(df):
    x_list= df['emb'].to_list()
    y_list =  df.label.to_list()
    # Generate batches
    while True:
        for b in range(batches_per_epoch_val):
            longest_index = (b + 1) * batch_size_val - 1
            timesteps = len(max(df['emb'].to_list()[:(b + 1) * batch_size_val][-31:], key=len))
            #print(len(df_train['emb'].to_list()[:b+batch_size][-7:]))
            x_train = np.full((batch_size_val, timesteps, num_features), -99.)
            y_train = np.zeros((batch_size_val,  1))
            for i in range(batch_size_val):
                li = b * batch_size_val + i
                #print("li", li)
                #print(x_train[i, 0:len(x_list[li]), :].shape, len(x_list[li]))
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            yield x_train, y_train

Sample Documents
(Place all the documents to be classified under /content)

In [45]:
!gdown --id 10KS_d5Yqaw2VbRQQ8SpZuGMwPZpvBWXw

Downloading...
From: https://drive.google.com/uc?id=10KS_d5Yqaw2VbRQQ8SpZuGMwPZpvBWXw
To: /content/Letter-Abdul Mannan Zafar.pdf
100% 103k/103k [00:00<00:00, 29.8MB/s]


In [46]:
!gdown --id 1tgQxek831nw_U4GTSiYBSF8VPaoBmDgQ

Downloading...
From: https://drive.google.com/uc?id=1tgQxek831nw_U4GTSiYBSF8VPaoBmDgQ
To: /content/Technical Seminar Synopsis - Abdul Mannan Zafar.pdf
100% 578k/578k [00:00<00:00, 38.5MB/s]


### Document Data Extraction

In [39]:
import os
import textract
import pandas as pd
import PyPDF2

# file to store the data
file=open("projeg.txt",'w')

#basepath is the present directory scanned
basepath = './'

doclst=[["name","filedata"]]

doc=["doc","docx","html","htm","txt"]

#printing the files scanned and the data is stored in the "file"
with os.scandir(basepath) as entries:
	for entry in entries:
		if entry.is_file():
			name=(entry.name).split(".")
			if name[1] in doc:
				doclst.append([(basepath + entry.name),((textract.process(basepath+entry.name)).decode())])
			if name[1] == "pdf":
				pagedata=""
				with open((basepath + entry.name),'rb') as pdf_file:
					read_pdf = PyPDF2.PdfFileReader(pdf_file)
					for page_number in range(read_pdf.getNumPages()):
						page = read_pdf.getPage(page_number)
						pagedata += page.extractText()
				doclst.append([(basepath + entry.name),(pagedata)])
for i in doclst:
	file.write("\n" + i[0] + "\n" + i[1])

In [40]:
fname=[]

### Confidential Document Prediction

In [41]:
pred_sentences= []
i=0
for x in doclst:
  pred_sentences=x
  pred_sentences[1]=pred_sentences[1].replace("\n"," ")
  print(pred_sentences[1])
  #pred_sentences.append(doclst[i])
  #pred_sentences.append(doclst[i+1])
  #i=i+2
  tst_label= [0]*len(pred_sentences)
  test = pd.DataFrame({'text':pred_sentences, 'label':tst_label, })
  test['text_split']=test["text"].apply(get_split)

  test_l = []
  test_label_l = []
  test_index_l = []
  for idx,row in test.iterrows():
    for l in row['text_split']:
      test_l.append(l)
      test_label_l.append(row['label'])
      test_index_l.append(idx)
  len(test_l), len(test_label_l), len(test_index_l)
  test_df = pd.DataFrame({DATA_COLUMN:test_l, LABEL_COLUMN:test_label_l})
  print(test_df.head())
  flag=0
  for x in imp:
    if(x in pred_sentences[1].lower()):
      flag=1
  #model = tf.compat.v2.saved_model.load(str('/content/bert_news_category/1596907431'), None)
  #  model = model.signatures["predict"]
  test_emb = np.apply_along_axis(getPrediction,0,np.array(test_df[DATA_COLUMN]),predict_fn)
  print(test_emb.shape)
  aux = -1
  len_l = 0
  test_x = {}

  for l, emb in zip(test_index_l, test_emb):
    if l in test_x.keys():
      test_x[l]  =np.vstack([test_x[l], emb])
    else:
      test_x[l] = [emb]


  test_l_final = []
  tlabel_l_final = []
  for k in test_x.keys():
    test_l_final.append(test_x[k])
    tlabel_l_final.append(test.loc[k]['label'])

  df_test = pd.DataFrame({'emb': test_l_final, 'label': tlabel_l_final})
  print(df_test.head())
  print(df_test.shape)
  num_sequences_val = len(df_test['emb'].to_list())
  batch_size_val = 1
  batches_per_epoch_val = 2
  assert batch_size_val * batches_per_epoch_val == num_sequences_val
  num_features= 768
  res1=model.predict_generator(val_generator(df_test), steps=batches_per_epoch_val)
  print(res1)
  lst=[]
  for x in res1:
    lst.append(x.tolist().index(max(x)))
  df_test['label']=lst
  conf=[0]*len(lst)

  for i in range(0,len(lst)):
    if(lst[i]==0 or lst[i]==2 or lst[i]==3 or lst[i]==5 or lst[i]==7 or flag==1):
      conf[i]=1
  if(conf[1]):
    fname.append(pred_sentences[0])
  df_test['confidentiality']=conf
  print(df_test)
  df = pd.DataFrame(None)
  lst.clear()
  conf.clear()


filedata
       text  label
0      name      0
1  filedata      0
INFO:tensorflow:Writing example 0 of 2


INFO:tensorflow:Writing example 0 of 2


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


[array([[-0.54559904, -0.6514454 , -0.2312796 , ..., -0.87588143,
        -0.44856656,  0.7901454 ],
       [-0.8345141 , -0.35685617,  0.8250994 , ...,  0.43714058,
         0.1113009 ,  0.90122503]], dtype=float32)]
(2, 768)
                                                 emb  label
0  [[-0.54559904, -0.6514454, -0.2312796, 0.66368...      0
1  [[-0.8345141, -0.35685617, 0.8250994, 0.838091...      0
(2, 2)
[[6.7042804e-04 1.0497753e-01 1.8855763e-03 2.3991134e-02 1.1905316e-01
  5.1501542e-03 8.5063078e-02 7.2259055e-03 4.6771517e-04 6.5151536e-01]
 [2.6370291e-02 1.8897098e-01 4.7629863e-02 7.8878626e-02 9.9185646e-02
  9.3427266e-04 5.3649002e-01 1.4814585e-02 1.7819608e-03 4.9438435e-03]]
                                                 emb  label  confidentiality
0  [[-0.54559904, -0.6514454, -0.2312796, 0.66368...      9                0
1  [[-0.8345141, -0.35685617, 0.8250994, 0.838091...      6                0


In [42]:
df_output=pd.DataFrame(columns=['conf_file'])

### List of Confidential Documents extracted.

In [43]:
df_output['conf_file']=fname
df_output

,conf_file
0,./Letter-Abdul Mannan Zafar.pdf
